# Création données géographiques

## csv vers graphe

On va charger le fichier des villes française puis créer un point avec les coordonnées. On va procéder comme dans le [tutoriel](https://geopandas.org/gallery/create_geopandas_from_pandas.html).

On utilise [geopandas.read_file](https://geopandas.org/docs/reference/api/geopandas.read_file.html#geopandas-read-file)

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
v1 = pd.read_csv("./villes_france_métropole_30000.csv")

In [ ]:
v1

In [ ]:
v1.dtypes

In [ ]:
v1.columns

Attention, le [code insee](https://fr.wikipedia.org/wiki/Code_officiel_g%C3%A9ographique) n'est pas le code postal.

In [ ]:
v1[lambda df: df["nom"] == "Marseille"]

In [ ]:
v1[lambda df: df.population >= 250000]

In [ ]:
len(v1[lambda df: df.population >= 100000])

In [ ]:
len(v1[lambda df: df.population >= 50000])

On crée la geometry en disant que c'est un point, et on place le crs à ce qu'il faut.

In [ ]:
villes = gpd.GeoDataFrame(v1, 
                          geometry=gpd.points_from_xy(v1.longitude, v1.latitude), 
                          crs="EPSG:4326")

In [ ]:
villes.plot()

garder les grandes villes :

In [ ]:
villes[lambda df: df.population >= 50000].plot()

Beaucoup de villes entre 50000 et 100000 habitants en ile de France.

In [ ]:
regions = gpd.read_file("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions.geojson")

In [ ]:
regions

In [ ]:
regions.loc[0]

In [ ]:
len(villes[lambda df: df.geometry.within(regions.loc[0].geometry)])

In [ ]:
villes[lambda df: df.population >= 50000]

In [ ]:
villes[lambda df: (df.population >= 50000) & (df.geometry.within(regions.loc[0].geometry))]

In [ ]:
len(villes[lambda df: (df.population >= 50000) & (df.geometry.within(regions.loc[0].geometry))])

## Carte interactive

In [ ]:
import folium

In [ ]:
from geopy import Nominatim # geodécodeur d'openstretmap

locator = Nominatim(user_agent="jupiter cours")
centre_Fr = locator.geocode("Bruère-Allichamps, France")

In [ ]:
m = folium.Map(location=centre_Fr.point[:2],
               #tiles='Stamen Terrain',
               zoom_start=5
              )

In [ ]:
m

In [ ]:
villes[lambda df: (df.population >= 200000)]

J'utilise le tuto de <https://geopandas.org/en/stable/gallery/plotting_with_folium.html#Add-markers> :

In [ ]:
for i, ville in villes[lambda df: (df.population >= 200000)].iterrows():
    folium.Marker(
        [ville.latitude, ville.longitude],
        icon=folium.Icon(color='green'),
        popup=ville.nom
    ).add_to(m)


In [ ]:


m